In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
order_products_train = pd.read_csv('Downloads/Data Files/order_products__train.csv')
order_products_prior = pd.read_csv('Downloads/Data Files/order_products__prior.csv')
orders      = pd.read_csv('Downloads/Data Files/orders.csv')
products    = pd.read_csv('Downloads/Data Files/products.csv')
aisles      = pd.read_csv('Downloads/Data Files/aisles.csv')
departments = pd.read_csv('Downloads/Data Files/departments.csv')

## Merge the tables

In [3]:
full_products = products.copy()
full_products = pd.merge(full_products, aisles,               on='aisle_id',      how='left')
full_products = pd.merge(full_products, departments,          on='department_id', how='left')
order_products_train = pd.merge(order_products_train, orders, on='order_id',      how='left')
order_products_prior = pd.merge(order_products_prior, orders, on='order_id',      how='left')
train         = pd.merge(order_products_train, full_products, on='product_id',    how='left')
test          = pd.merge(order_products_prior, full_products, on='product_id',    how='left')

In [4]:
train.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
4,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods


In [5]:
print(f'train.shape: {train.shape}')
print(f'test.shape : {test.shape}')

train.shape: (1384617, 15)
test.shape : (32434489, 15)


In [6]:
train.describe()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
count,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06
mean,1.706298e+06,2.555624e+04,8.758044e+00,5.985944e-01,1.031128e+05,1.709141e+01,2.701392e+00,1.357759e+01,1.706613e+01,7.130423e+01,9.839777e+00
std,9.897326e+05,1.412127e+04,7.423936e+00,4.901829e-01,5.948715e+04,1.661404e+01,2.167646e+00,4.238458e+00,1.042642e+01,3.810409e+01,6.292830e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,8.433700e+05,1.338000e+04,3.000000e+00,0.000000e+00,5.173200e+04,6.000000e+00,1.000000e+00,1.000000e+01,7.000000e+00,3.100000e+01,4.000000e+00
50%,1.701880e+06,2.529800e+04,7.000000e+00,1.000000e+00,1.029330e+05,1.100000e+01,3.000000e+00,1.400000e+01,1.500000e+01,8.300000e+01,8.000000e+00
75%,2.568023e+06,3.794000e+04,1.200000e+01,1.000000e+00,1.549590e+05,2.100000e+01,5.000000e+00,1.700000e+01,3.000000e+01,1.070000e+02,1.600000e+01
max,3.421070e+06,4.968800e+04,8.000000e+01,1.000000e+00,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01,1.340000e+02,2.100000e+01


## Deal with missing values

In [7]:
train = train.sample(frac=0.5)

In [8]:
test = test.sample(frac=0.1)

In [9]:
print(f'train.shape: {train.shape}')
print(f'test.shape : {test.shape}')

train.shape: (692308, 15)
test.shape : (3243449, 15)


In [10]:
def fillna(df):
    numerical_features = df.select_dtypes(include=[np.number]).columns
    df[numerical_features] = df[numerical_features].fillna(df[numerical_features].median())
    categorical_features = df.select_dtypes(exclude=[np.number]).columns
    for col in categorical_features:
        df[col] = df[col].fillna(df[col].mode()[0])

def replace_inf(df):
    numerical_features = df.select_dtypes(include=[np.number]).columns
    df[numerical_features] = df[numerical_features].replace([np.inf, -np.inf], np.nan)
    df[numerical_features] = df[numerical_features].fillna(df[numerical_features].median())

In [11]:
replace_inf(train)
replace_inf(test)
fillna(train)
fillna(test)

In [12]:
train.describe()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
count,6.923080e+05,692308.000000,692308.000000,692308.000000,692308.000000,692308.000000,692308.000000,692308.000000,692308.000000,692308.000000,692308.000000
mean,1.707456e+06,25545.602811,8.744527,0.599116,103068.182595,17.086922,2.703801,13.582144,17.049880,71.323600,9.836693
std,9.900543e+05,14119.547157,7.407811,0.490078,59474.924243,16.607877,2.168477,4.236584,10.428404,38.111941,6.294736
min,1.000000e+00,1.000000,1.000000,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,8.449330e+05,13351.000000,3.000000,0.000000,51704.000000,6.000000,1.000000,11.000000,7.000000,31.000000,4.000000
50%,1.703508e+06,25272.000000,7.000000,1.000000,102936.500000,11.000000,3.000000,14.000000,15.000000,83.000000,8.000000
75%,2.570848e+06,37936.000000,12.000000,1.000000,154814.000000,21.000000,5.000000,17.000000,30.000000,107.000000,16.000000
max,3.421070e+06,49688.000000,80.000000,1.000000,206209.000000,100.000000,6.000000,23.000000,30.000000,134.000000,21.000000


In [13]:
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [14]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = order_products_prior.set_index('order_id')['product_id'].rename('item_id')
display(orders.head(10))
type(orders)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: item_id, dtype: int64

pandas.core.series.Series

# ASSOCIATION RULE LEARNING

In [15]:
## The below functions generate association rules from transaction data by calculating item frequencies, filtering based on support thresholds, 
## and computing metrics for item pairs. The result is a DataFrame of association rules sorted by their lift, which helps identify strong and 
## interesting relationships between items.

In [16]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]    

In [17]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [18]:
rules = association_rules(orders, 0.01) 

Starting order_item:               32434489
Items with support >= 0.01:           10906
Remaining order_item:              29843570
Remaining orders with 2+ items:     3013325
Remaining order_item:              29662716
Item pairs:                        30622410
Item pairs with support >= 0.01:      48751



In [19]:
item_name   = products.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,306,0.010155,1163,0.038595,839,0.027843,0.263113,0.364720,9.449868
1,Grain Free Chicken Formula Cat Food,Grain Free Turkey Formula Cat Food,318,0.010553,1809,0.060033,879,0.029170,0.175788,0.361775,6.026229
3,Organic Fruit Yogurt Smoothie Mixed Berry,Apple Blueberry Fruit Yogurt Smoothie,349,0.011582,1518,0.050376,1249,0.041449,0.229908,0.279424,5.546732
9,Nonfat Strawberry With Fruit On The Bottom Gre...,"0% Greek, Blueberry on the Bottom Yogurt",409,0.013573,1666,0.055288,1391,0.046162,0.245498,0.294033,5.318230
10,Organic Grapefruit Ginger Sparkling Yerba Mate,Cranberry Pomegranate Sparkling Yerba Mate,351,0.011648,1731,0.057445,1149,0.038131,0.202773,0.305483,5.317849
...,...,...,...,...,...,...,...,...,...,...,...
7271,Organic Strawberries,Strawberries,640,0.021239,263416,8.741706,141805,4.705931,0.002430,0.004513,0.000516
6763,Organic Hass Avocado,Organic Avocado,464,0.015398,212785,7.061469,176241,5.848722,0.002181,0.002633,0.000373
4387,Organic Avocado,Organic Hass Avocado,443,0.014701,176241,5.848722,212785,7.061469,0.002514,0.002082,0.000356
2596,Banana,Bag of Organic Bananas,654,0.021704,470096,15.600574,376367,12.490090,0.001391,0.001738,0.000111
